In [1]:
import pandas as pd
import json
from collections import defaultdict

In [2]:
cities_tables = defaultdict(list)
with open('../../data/tabelas/tabelas_atas_v3.json') as file:
    ata_files = json.load(file)
    for city, files in ata_files.items():
        for file in files:
            cities_tables[city].append(file['id'].split('.')[0])

sicom_result = pd.read_csv('../../data/tabelas/atas_sicom.csv')
# sicom_result['index'] = sicom_result[['nome_cidade_show', 'ano_referencia', 'num_processo']].apply(lambda row: '/'.join(row.values.astype(str)), axis=1)
# sicom_result.set_index('index', inplace=True)

In [3]:
def get_process_num_year(info):
    num, year = info.split('/')
    return str(int(num)), year

In [4]:
cities = ['arantina', 'coqueiral', 'cristais', 'ijaci', 'olaria', 'passa-vinte', 'pirapetinga', 'sao-bento-abade']
# cities = ['coqueiral']
file_paths = [f'../../data/resultado_processo_licitatorio/{city}.csv' for city in cities]
cities_atas = defaultdict(lambda: defaultdict(list))
for path in file_paths:
    city = path.split('/')[-1].split('.')[0]
    print(f'## {city} ##')
    df = pd.read_csv(path)
    df['arquivos'] = df['arquivos'].str.split(',')
    print(f'Total de arquivos de lances:', df.shape[0])
    print(f'Total de atas com tabelas:', len(cities_tables[city]))
    count = 0
    for num_process, row in df.iterrows():
        files = []
        for file in row['arquivos']:
            if file in cities_tables[city]:
                files.append(file)
                
        if len(files) > 0:
            num, year = get_process_num_year(row['no. licitação'])
            cities_atas[city][year].append((num, files))

    if len(cities_tables[city]) > 0:
        print(f'Total de atas com lances: {len(cities_atas[city])} ({round((len(cities_atas[city])/len(cities_tables[city]))*100, 2)}%)')
    else:
        print(f'Total de atas com lances: {len(cities_atas[city])} (0%)')

## arantina ##
Total de arquivos de lances: 220
Total de atas com tabelas: 0
Total de atas com lances: 0 (0%)
## coqueiral ##
Total de arquivos de lances: 390
Total de atas com tabelas: 295
Total de atas com lances: 4 (1.36%)
## cristais ##
Total de arquivos de lances: 286
Total de atas com tabelas: 356
Total de atas com lances: 3 (0.84%)
## ijaci ##
Total de arquivos de lances: 176
Total de atas com tabelas: 0
Total de atas com lances: 0 (0%)
## olaria ##
Total de arquivos de lances: 15
Total de atas com tabelas: 0
Total de atas com lances: 0 (0%)
## passa-vinte ##
Total de arquivos de lances: 50
Total de atas com tabelas: 7
Total de atas com lances: 1 (14.29%)
## pirapetinga ##
Total de arquivos de lances: 162
Total de atas com tabelas: 67
Total de atas com lances: 3 (4.48%)
## sao-bento-abade ##
Total de arquivos de lances: 139
Total de atas com tabelas: 47
Total de atas com lances: 2 (4.26%)


In [5]:
sicom_result.columns

Index(['ano_referencia', 'num_processo', 'metadata_arquivo_linha',
       'id_julgamento', 'id_item_licitacao', 'id_licitacao', 'id_orgao',
       'cod_orgao', 'nome_orgao_show', 'cod_nome_tipo_orgao_show',
       'nome_tipo_orgao_show', 'nome_regiaoplanejamento_show', 'ano_exercicio',
       'mes_licitacao', 'num_lote_raw', 'num_lote_show', 'nome_lote_raw',
       'nome_lote_show', 'num_item_raw', 'num_item_show', 'nome_item_raw',
       'nome_item_show', 'vlr_unitario_raw', 'vlr_unitario_show',
       'num_quant_item_raw', 'num_quant_item_show', 'vlr_perc_desconto_raw',
       'vlr_perc_desconto_show', 'vlr_perc_taxa_adm_raw',
       'vlr_perc_taxa_adm_show', 'vlr_global_raw', 'vlr_global_show',
       'dsc_ind_desonera_folha_raw', 'dsc_ind_desonera_folha_show',
       'cod_ibge_municipio', 'num_cpf_cnpj_municipio', 'cod_siafi',
       'nome_cidade_show', 'ibge_microrregiao_id_microrregiao',
       'nome_microrregiao_show', 'ibge_mesorregiao_id_mesorregiao',
       'nome_mesorregiao_

In [12]:
query_base = str("select DISTINCT A.num_processo from dataset_v2.sicom_20210109_licitacao as A " 
            + "join dataset_v2.sicom_20210109_licitacao_julgamento as B "
            + "on A.id_licitacao = B.id_licitacao where ")
where_clause = str("(A.ano_referencia = '_YEAR_' and A.nome_cidade_show = '_CITY_' and A.num_processo IN _PROCESS_LIST_)")

for city, files in cities_atas.items():
    city = city.replace('-', ' ').upper()
    print(f'## {city} ##')
    for year, tuple_list in files.items():
        nums = ()
        for t in tuple_list:
#             print('city: ', city)
#             print('year: ', year)
#             print('t[0]: ', t[0])
#             print('t[1]: ', t[1])
#             print(f'{city}/{year}/{t[0]}')
            nums += (t[0],)
            index = sicom_result[(sicom_result.ano_referencia == int(year)) & 
                            (sicom_result.nome_cidade_show == city) & 
                            (sicom_result.num_processo == int(t[0]))].index
#             print('index: ', index.values)
            sicom_result.at[index, 'files'] = pd.Series(t[1])
        query_base += where_clause.replace('_YEAR_', year).replace('_CITY_', city).replace('_PROCESS_LIST_', str(nums)) + ' or '
#         print(where_clause.replace('_YEAR_', year).replace('_CITY_', city).replace('_PROCESS_LIST_', str(nums)))
#         print('-'*50)
query_base

## ARANTINA ##
## COQUEIRAL ##
## CRISTAIS ##
## IJACI ##
## OLARIA ##
## PASSA VINTE ##
## PIRAPETINGA ##
## SAO BENTO ABADE ##


"select DISTINCT A.num_processo from dataset_v2.sicom_20210109_licitacao as A join dataset_v2.sicom_20210109_licitacao_julgamento as B on A.id_licitacao = B.id_licitacao where (A.ano_referencia = '2017' and A.nome_cidade_show = 'COQUEIRAL' and A.num_processo IN ('132',)) or (A.ano_referencia = '2018' and A.nome_cidade_show = 'COQUEIRAL' and A.num_processo IN ('15', '55', '86', '87', '106', '119', '122', '126', '135', '139', '143', '144', '150', '156', '161', '172', '180', '187', '201')) or (A.ano_referencia = '2019' and A.nome_cidade_show = 'COQUEIRAL' and A.num_processo IN ('6', '8', '11', '35', '48', '60', '63', '80', '96', '108', '112')) or (A.ano_referencia = '2020' and A.nome_cidade_show = 'COQUEIRAL' and A.num_processo IN ('2', '19', '27', '31', '43', '82', '85', '86', '87', '88', '102', '107')) or (A.ano_referencia = '2015' and A.nome_cidade_show = 'CRISTAIS' and A.num_processo IN ('32',)) or (A.ano_referencia = '2016' and A.nome_cidade_show = 'CRISTAIS' and A.num_processo IN ('

In [10]:
sicom_result = sicom_result[['ano_referencia', 'num_processo', 'metadata_arquivo_linha', 'nome_cidade_show', 'files',
       'id_julgamento', 'id_item_licitacao', 'id_licitacao', 'id_orgao',
       'cod_orgao', 'nome_orgao_show', 'cod_nome_tipo_orgao_show',
       'nome_tipo_orgao_show', 'nome_regiaoplanejamento_show', 'ano_exercicio',
       'mes_licitacao', 'num_lote_raw', 'num_lote_show', 'nome_lote_raw',
       'nome_lote_show', 'num_item_raw', 'num_item_show', 'nome_item_raw',
       'nome_item_show', 'vlr_unitario_raw', 'vlr_unitario_show',
       'num_quant_item_raw', 'num_quant_item_show', 'vlr_perc_desconto_raw',
       'vlr_perc_desconto_show', 'vlr_perc_taxa_adm_raw',
       'vlr_perc_taxa_adm_show', 'vlr_global_raw', 'vlr_global_show',
       'dsc_ind_desonera_folha_raw', 'dsc_ind_desonera_folha_show',
       'cod_ibge_municipio', 'num_cpf_cnpj_municipio', 'cod_siafi',
        'ibge_microrregiao_id_microrregiao',
       'nome_microrregiao_show', 'ibge_mesorregiao_id_mesorregiao',
       'nome_mesorregiao_show', 'ibge_regiao_imediata_id_regiao_imediata',
       'nome_regiao_imediata_show',
       'ibge_regiao_intermediaria_id_regiao_intermediaria',
       'nome_regiao_intermediaria_show', 'id_uf', 'sigla_uf', 'longitude',
       'latitude', 'num_versao_arq', 'ano_arquivo', 'metadata_data_referencia',
       'metadata_arquivo_original_zip',
       'metadata_arquivo_original_zip_md5hash', 'metadata_arquivo_original',
       'metadata_arquivo_original_linha', 'metadata_arquivo',
       'metadata_data_ingestao']]

sicom_result.to_csv('../../data/tabelas/atas_sicom_v2.csv')

In [13]:
{'ano_referencia': 'first', 'num_processo': 'first', 'metadata_arquivo_linha',
       'nome_cidade_show', 'files', 'id_julgamento', 'id_item_licitacao',
       'id_licitacao', 'id_orgao', 'cod_orgao', 'nome_orgao_show',
       'cod_nome_tipo_orgao_show', 'nome_tipo_orgao_show',
       'nome_regiaoplanejamento_show', 'ano_exercicio', 'mes_licitacao',
       'num_lote_raw', 'num_lote_show', 'nome_lote_raw', 'nome_lote_show',
       'num_item_raw', 'num_item_show', 'nome_item_raw', 'nome_item_show',
       'vlr_unitario_raw', 'vlr_unitario_show', 'num_quant_item_raw',
       'num_quant_item_show', 'vlr_perc_desconto_raw',}
sicom_result.columns

SyntaxError: invalid syntax (<ipython-input-13-e5a7db031225>, line 1)